In [ ]:
#| default_exp deepreviewer

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import os, tiktoken, numpy as np, multiprocessing as mp, pandas as pd, json
from datasets import load_dataset
from tqdm.auto import tqdm

In [ ]:
#| export
def write_datafile(fname, tokens):
    np.save(fname, tokens)
    

In [ ]:
filename = '/home/scai/phd/aiz218323/scratch/datasets/deepreviewer/train.csv'
df = pd.read_csv(filename)

docs = [json.loads(df.iloc[i, 0])[1]['content'] for i in range(df.shape[0])]

In [ ]:
shard_size = int(1e8)

enc = tiktoken.get_encoding('gpt2')
eot = enc._special_tokens['<|endoftext|>']

In [ ]:
def tokenize(doc):
    tokens = [eot]
    tokens.extend(enc.encode_ordinary(doc))
    tokens_np = np.array(tokens)
    assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), 'token dictionary too large  for uint16'
    tokens_np_uint16 = tokens_np.astype(np.uint16)
    return tokens_np_uint16
    

In [ ]:
o = tokenize(docs[0]); o

array([50256,    59,  7839, ...,   628,   628,   198], dtype=uint16)

In [ ]:
cache_dir = '/home/scai/phd/aiz218323/scratch/datasets/deepreviewer/cache'
os.makedirs(cache_dir, exist_ok=True)

n_procs = max(1, os.cpu_count()//2)

In [ ]:
with mp.Pool(n_procs) as pool:
    shard_index,token_count = 0,0
    all_tokens_np = np.empty(shard_size, dtype=np.uint16)
    
    progress_bar = None
    for tokens in pool.imap(tokenize, docs, chunksize=16):

        if token_count + len(tokens) > shard_size:
            all_tokens_np[token_count:token_count+len(tokens)] = tokens
            token_count += len(tokens)
            if progress_bar is None:
                progress_bar = tqdm(total=shard_size, unit="tokens", desc=f"Shard {shard_index}")
            progress_bar.update(len(tokens))
        else:
            split = "val" if shard_index == 0 else "train"
            fname = os.path.join(cache_dir, f"deepreviewer_{split}_{shard_index:06d}")
            
            remainder = shard_size - token_count
            all_tokens_np[token_count:] = tokens[:remainder]
            progress_bar.update(remainder)

            
            write_datafile(fname, all_tokens_np)
            shard_index += 1
            progress_bar = None

            token_count = len(tokens) - remainder
            all_tokens_np[:token_count] = tokens[remainder:]
            

In [ ]:
if token_count != 0:
    split = "val" if shard_index == 0 else "train"
    fname = os.path.join(cache_dir, f"deepreviewer_{split}_{shard_index:06d}")
    write_datafile(fname, all_tokens_npp[:token_count])

## `__main__`

In [ ]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--filename', type=str, required=True)
    parser.add_argument('--cache_dir', type=str, required=True)
    return parser.parse_args()
    

In [ ]:
#| export
if __name__ == "__main__":
    args = parse_args()

    df = pd.read_csv(args.filename)
    docs = [json.loads(df.iloc[i, 0])[1]['content'] for i in range(df.shape[0])]

    shard_size = int(1e8)

    enc = tiktoken.get_encoding('gpt2')
    eot = enc._special_tokens['<|endoftext|>']

    def tokenize(doc):
        tokens = [eot]
        tokens.extend(enc.encode_ordinary(doc))
        tokens_np = np.array(tokens)
        assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), 'token dictionary too large  for uint16'
        tokens_np_uint16 = tokens_np.astype(np.uint16)
        return tokens_np_uint16

    os.makedirs(args.cache_dir, exist_ok=True)

    n_procs = max(1, os.cpu_count()//2)
    with mp.Pool(n_procs) as pool:
        shard_index,token_count = 0,0
        all_tokens_np = np.empty(shard_size, dtype=np.uint16)
        
        progress_bar = None
        for tokens in pool.imap(tokenize, docs, chunksize=16):
    
            if token_count + len(tokens) > shard_size:
                all_tokens_np[token_count:token_count+len(tokens)] = tokens
                token_count += len(tokens)
                if progress_bar is None:
                    progress_bar = tqdm(total=shard_size, unit="tokens", desc=f"Shard {shard_index}")
                progress_bar.update(len(tokens))
            else:
                split = "val" if shard_index == 0 else "train"
                fname = os.path.join(args.cache_dir, f"deepreviewer_{split}_{shard_index:06d}")
                
                remainder = shard_size - token_count
                all_tokens_np[token_count:] = tokens[:remainder]
                progress_bar.update(remainder)
    
                
                write_datafile(fname, all_tokens_np)
                shard_index += 1
                progress_bar = None
    
                token_count = len(tokens) - remainder
                all_tokens_np[:token_count] = tokens[remainder:]

    if token_count != 0:
        split = "val" if shard_index == 0 else "train"
        fname = os.path.join(args.cache_dir, f"deepreviewer_{split}_{shard_index:06d}")
        write_datafile(fname, all_tokens_npp[:token_count])
    